In [11]:
import pandas as pd
import sklearn
import torch
import numpy as np
from  sklearn.model_selection import train_test_split
import torch.nn as nn

In [6]:
# train_data = pd.read_csv("bit-x-adata/train.csv")
# train_data.drop(columns=["id"], inplace=True)
# train_data[["święto", "dzień_roboczy"]] = train_data[["święto", "dzień_roboczy"]].astype(int)
# train_data[['year', 'month', 'day']] = train_data['data'].str.split('-', expand=True)
# train_data.drop("data", axis=1, inplace=True)

# test_data = pd.read_csv("bit-x-adata/test.csv")
# test_data.drop(columns=["id"], inplace=True)
# test_data[["święto", "dzień_roboczy"]] = test_data[["święto", "dzień_roboczy"]].astype(int)
# test_data[['year', 'month', 'day']] = test_data['data'].str.split('-', expand=True)
# test_data.drop("data", axis=1, inplace=True)

# train_data_with_dummies = pd.get_dummies(train_data, columns=['pogoda'], dtype=int)
# test_data_with_dummies = pd.get_dummies(test_data, columns=['pogoda'], dtype=int)

# X_train = train_data.drop("studenty_ms", axis=1).values
# X_train_with_dummies = train_data_with_dummies.drop("studenty_ms", axis=1).values

# y_train = train_data["studenty_ms"].values.reshape(-1,1)

# X_test = test_data.values


In [7]:
train_data = pd.read_csv("bit-x-adata/train.csv")
train_data.drop(columns=["id"], inplace=True)
train_data[["święto", "dzień_roboczy"]] = train_data[["święto", "dzień_roboczy"]].astype(int)
train_data[['year', 'month', 'day']] = train_data['data'].str.split('-', expand=True)
train_data.drop("data", axis=1, inplace=True)

train_data_with_dummies = pd.get_dummies(train_data, columns=['pogoda'], dtype=int)
X = train_data_with_dummies.drop("studenty_ms", axis=1).values
Y = train_data_with_dummies["studenty_ms"].values.reshape(-1,1)
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=42)


In [8]:
train_data_with_dummies.head()

,święto,dzień_roboczy,temperatura,odczuwalna_temperatura,wilgotność,prędkość_wiatru,studenty_ms,year,month,day,pogoda_lekkie_opady,pogoda_pochmurno,pogoda_ładna_pogoda
0,0,1,2,0,39,23,12,2020,01,02,0,0,1
1,0,1,-2,0,47,7,15,2020,01,03,0,0,1
2,0,0,-1,-3,47,11,6,2020,01,05,0,0,1
3,1,0,-2,-4,51,11,18,2020,01,06,0,0,1
4,0,1,-6,-7,36,16,11,2020,01,08,0,0,1


In [9]:
X_train.shape[1]

12

In [12]:
class GepardPred(nn.Module):
    def __init__(self, input_size: int, dropout_rate: float = 0.5):
        super().__init__()

        self.model = nn.Sequential(
            nn.Linear(input_size, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 256),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.Linear(256, 1)
        )

    def forward(self, x):
        return self.model(x)

In [13]:
from torch import optim

In [14]:
class RMSLELoss(nn.Module):
    def __init__(self):
        super().__init__()
        self.mse = nn.MSELoss()

    def forward(self, pred, actual):
        return torch.sqrt(self.mse(torch.log(pred + 1), torch.log(actual + 1)))

In [15]:
from torch.utils.data import Dataset


class MyDataset(Dataset):
    def __init__(self, data, y):
        super().__init__()

        self.data = data
        self.y = y

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.data[idx], self.y[idx]



In [16]:
from torch.utils.data import DataLoader

learning_rate = 1e-3
dropout_p = 0.5
l2_reg = 1e-4
batch_size = 128
max_epochs = 300

early_stopping_patience = 4

In [18]:
from copy import deepcopy

In [ ]:
def evaluate_model(model, X, y, loss_fn):
    model.eval()
    with torch.no_grad():
        inputs = torch.tensor(X, dtype=torch.float32)
        targets = torch.tensor(y, dtype=torch.float32)

        outputs = model(inputs)
        loss = loss_fn(outputs, targets)

    return {"loss": loss.item()}

In [ ]:
dropout_p = 0.3
learning_rate = 1e-3

model = GepardPred(
    input_size=X_train.shape[1],
    dropout_rate=dropout_p
)

optimizer = optim.AdamW(
    model.parameters(),
    lr=learning_rate,
    weight_decay=l2_reg
)

train_dataset = MyDataset(
    X_train,
    y_train
)

data_loader = DataLoader(
    train_dataset,
    batch_size
)

loss_fn = RMSLELoss()

best_model = model
best_threshold = None
patience = 4
steps_without_improvement = 0
best_loss = np.inf

for epoch in range(max_epochs):

    model.train()

    for X_batch, y_batch in data_loader:
        y_pred = model(X_batch)

        loss = loss_fn(y_pred, y_batch)
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

    valid_metrics = evaluate_model(model, X_valid, y_valid, loss_fn)

    valid_loss = valid_metrics["loss"]

    print(f"Epoch {epoch} train loss: {loss.item():.4f}, eval loss {valid_loss:.4f}")

    if valid_loss < best_loss:
        best_loss = valid_loss
        best_model = deepcopy(model)

        steps_without_improvement = 0
    else:
        steps_without_improvement += 1
        if steps_without_improvement >= patience:

            print(f"Early stopping at epoch {epoch}")
            break



KeyboardInterrupt

